# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
# from unityagents import UnityEnvironment
# import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [2]:
# env = UnityEnvironment(file_name="./Tennis_Linux/Tennis.x86_64")

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# # get the default brain
# brain_name = env.brain_names[0]
# brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [4]:
# # reset the environment
# env_info = env.reset(train_mode=True)[brain_name]

# # number of agents 
# num_agents = len(env_info.agents)
# print('Number of agents:', num_agents)

# # size of each action
# action_size = brain.vector_action_space_size
# print('Size of each action:', action_size)

# # examine the state space 
# states = env_info.vector_observations
# state_size = states.shape[1]
# print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
# print('The state for the first agent looks like:', states[0])

In [5]:
# states.shape

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [6]:
# for i in range(1, 6):                                      # play game for 5 episodes
#     env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
#     states = env_info.vector_observations                  # get the current state (for each agent)
#     scores = np.zeros(num_agents)                          # initialize the score (for each agent)
#     while True:
#         actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
#         actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
#         env_info = env.step(actions)[brain_name]           # send all actions to tne environment
#         next_states = env_info.vector_observations         # get next state (for each agent)
#         rewards = env_info.rewards                         # get reward (for each agent)
#         dones = env_info.local_done                        # see if episode finished
#         scores += env_info.rewards                         # update the score (for each agent)
#         states = next_states                               # roll over states to next time step
        
# #         print(len(actions), len(next_states), len(rewards), len(dones))
# #         print((next_states), (rewards), (dones))
# #         break
#         if np.any(dones):                                  # exit loop if episode finished
#             break
#     print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

When finished, you can close the environment.

In [7]:
# env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [8]:

import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

from unityagents import UnityEnvironment

from ddpg_agent import Agent, ReplayBuffer


In [9]:

class TennisMAEnv():
    def __init__(self, env_path:str="./Tennis_Linux/Tennis.x86_64"):
        self.env = UnityEnvironment(file_name=env_path)
        self.brain_name = self.env.brain_names[0]
        self.brain = self.env.brains[self.brain_name]
        self.action_size = self.brain.vector_action_space_size
        
    def reset(self, train_mode=True):
        env_info = self.env.reset(train_mode=train_mode)[self.brain_name]
        states = env_info.vector_observations
        return states
    
    def step(self, actions=None):
        if actions is None:
            actions=np.random.randn(2, 2)
            actions = np.clip(actions, -1, 1)
        env_info = self.env.step(actions)[self.brain_name]
        states = env_info.vector_observations
        rewards = env_info.rewards
        dones = env_info.local_done
        return states, rewards, dones, None
    
    def close(self):
        self.env.close()

env = TennisMAEnv()

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


In [10]:
agent1 = Agent(state_size=24, action_size=2, random_seed=2)
agent2 = Agent(state_size=24, action_size=2, random_seed=2)

In [ ]:
def MAddpg(n_episodes=9000, max_t=350, print_every=300):
    memory = ReplayBuffer(2, int(1e7), 128, 2)
    scores_deque = deque(maxlen=100)
    scores = []
    for i_episode in range(1, n_episodes+1):
        state = env.reset()
        agent1.reset()
        agent2.reset()
        score = np.zeros(2)
        while True:#for t in range(max_t):
            for i in range(2):
                action1 = agent1.act(state[0])
                action2 = agent2.act(state[1])
            action = [action1, action2]
            next_state, reward, done, _ = env.step(action)
            for i in range(2):
                memory = agent1.step(state[0], action[0], reward[0], next_state[0], done[0], memory)
                memory = agent2.step(state[1], action[1], reward[1], next_state[1], done[1], memory)
            state = next_state
            score += reward
            if any(done):
                break
        scores_deque.append(np.max(score))
        scores.append(np.max(score))
        print('\rEpisode {}\tAverage Score: {:.6f}\tMax Score: {:.6f}'.format(i_episode, np.mean(scores_deque), np.max(scores_deque)), end="")
        torch.save(agent1.actor_local.state_dict(), 'checkpoint_actor_ma1.pth')
        torch.save(agent1.critic_local.state_dict(), 'checkpoint_critic_ma1.pth')
        torch.save(agent2.actor_local.state_dict(), 'checkpoint_actor_ma2.pth')
        torch.save(agent2.critic_local.state_dict(), 'checkpoint_critic_ma2.pth')
        if i_episode % print_every == 0:
            print('\rEpisode {}\tAverage Score: {:.6f}\tMax Score: {:.6f}'.format(i_episode, np.mean(scores_deque), np.max(scores_deque)))
        if np.mean(scores_deque)>0.5:
            torch.save(agent1.actor_local.state_dict(), 'checkpoint_actor_ma1_0_5.pth')
            torch.save(agent1.critic_local.state_dict(), 'checkpoint_critic_ma1_0_5.pth')
            torch.save(agent2.actor_local.state_dict(), 'checkpoint_actor_ma2_0_5.pth')
            torch.save(agent2.critic_local.state_dict(), 'checkpoint_critic_ma2_0_5.pth')
#             break
            
    return scores

scores = MAddpg()


Episode 300	Average Score: 0.000000	Max Score: 0.000000
Episode 600	Average Score: 0.000000	Max Score: 0.000000
Episode 900	Average Score: 0.000000	Max Score: 0.000000
Episode 1200	Average Score: 0.000000	Max Score: 0.000000
Episode 1350	Average Score: 0.000000	Max Score: 0.000000

In [ ]:

# fig = plt.figure()
# ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()


In [ ]:
ma_scores = [np.mean(scores[i-100:i]) for i in range(100,len(scores))]
plt.plot(np.arange(1, len(ma_scores)+1), ma_scores)
plt.ylabel('Moving average Score over 100 episodes')
plt.xlabel('Episode #')
plt.show()

In [ ]:
agent.actor_local.load_state_dict(torch.load('checkpoint_actor.pth'))
agent.critic_local.load_state_dict(torch.load('checkpoint_critic.pth'))

state = env.reset(train_mode=False)
while True:
    action = agent.act(state, add_noise=False)
#     env.render()
    state, reward, done, _ = env.step(action)
    if done:
        break

env.close()

### 